In [ ]:
import os
import sys
import pandas as pd
import pathlib
import seaborn as sns
from matplotlib import pyplot as plt
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.conversions import angle_axis_to_rotation_matrix
import numpy as np
import matplotlib.pyplot as plt
from data_loader.data_loaders import SingleDataset
from model.deepvo.deepvo_model import DeepVOModel
from utils.conversions import se3_exp_map, quaternion_to_rotation_matrix
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# read trajectory files from csv
def read_trajectory_from_csv(file_name = None):
    '''Reads the trajectory of the robot in 3D space
    Args:
        file_name (str): name of the file to read
        '''    
    trajectory = pd.read_csv(os.path.join(os.getcwd(),file_name))
    return trajectory

def plot_route(trajectories = None, models = None,labels = None, colors = None):
    '''Plots the trajectory of the robot in 3D space
    Args:
        trajectories (list): list of trajectories to plot
        labels (list): list of labels for each trajectory,
        colors (list): list of colors for each trajectory
        '''    
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    ax1 = fig.add_subplot(231)
    ax2 = fig.add_subplot(234)
    ax3 = fig.add_subplot(232)
    for trajectory, label, color in zip(trajectories, models, colors):
        x = trajectory["x"]
        y = trajectory["y"]
        z = trajectory["z"]
        a1 = ax1.plot(x, y, color=color, label=label)
        plt.xlabel("X")
        plt.ylabel("Y")
        a2 = ax2.plot(x, z, color=color, label=label)
        plt.xlabel("X")
        plt.ylabel("Z")
        a3 = ax3.plot(y, z, color=color, label=label)
        plt.xlabel("Y")
        plt.ylabel("Z")
    fig.legend([a1, a2, a3],     # The line objects
           labels=labels,   # The labels for each line
           loc="lower right",   # Position of legend
           borderaxespad=0.1,    # Small spacing around legend box
           title="Legend Title"  # Title for the legend
           )
    plt.show()

def export_legend(legend, filename=None, expand=[-5,-5,5,5]):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
    print("legend.pdf")
    fig.savefig("legend.pdf", dpi="figure", bbox_inches=bbox)

def plot_route_XZ(trajectories = None, title = None, models = None,labels = None, colors = None):
    '''Plots the trajectory of the robot in 3D space
    Args:
        trajectories (list): list of trajectories to plot
        labels (list): list of labels for each trajectory,
        colors (list): list of colors for each trajectory
        '''    
    sns.set_style("ticks")
    # specify the custom font to us
    plt.rcParams['font.family'] = 'Latin Modern Roman'
    plt.rcParams['font.sans-serif'] = 'Latin Modern Roman'
    plt.rcParams['font.size'] = 30
    fig = plt.figure()
    ax2 = fig.add_subplot(111)
    for trajectory, label, color in zip(trajectories, labels, colors):
        x = trajectory["x"]
        y = trajectory["y"]
        z = trajectory["z"]

        a2 = ax2.plot(x, z, color=color, label=label)
        plt.xlabel("X")
        plt.ylabel("Z")
    legend = ax2.legend(bbox_to_anchor = (1.6, 0.82), loc='best', ncol=5, fontsize=15)
    export_legend(legend, title)
    ax2.set_title(title)
    plt.savefig(os.path.join(os.getcwd(),"..","media",title+'_XZ.pdf'), bbox_inches = 'tight') 
    plt.show()
    
def plot_route_YZ(trajectories = None, models = None,labels = None, colors = None):
    '''Plots the trajectory of the robot in 3D space
        Args:
        trajectories (list): list of trajectories to plot
        labels (list): list of labels for each trajectory,
        colors (list): list of colors for each trajectory
        '''    
    sns.set_style("ticks")
    # specify the custom font to us
    plt.rcParams['font.family'] = 'Latin Modern Roman'
    plt.rcParams['font.sans-serif'] = 'Latin Modern Roman'
    fig = plt.figure()
    ax3 = fig.add_subplot(111)
    for trajectory, label, color in zip(trajectories, models, colors):
        x = trajectory["x"]
        y = trajectory["y"]
        z = trajectory["z"]
        a3 = ax3.plot(y, z, color=color, label=label)
        plt.xlabel("Y")
        plt.ylabel("Z")
    ax3.legend(bbox_to_anchor = (1.25, 0.6),loc="upper right")
    plt.show()

In [ ]:
kitti_trajectories = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
models = ["target","deepvo", "deepvo_se3", "deepvo_quat", "deepvo_quat_geodesic"]
experiment = 'icra23'
# models = ["target","deepvo", "deepvo_se3"]
trajs = []
for traj_name in kitti_trajectories:
    for model in models:
        traj = read_trajectory_from_csv(file_name = experiment+'/'+traj_name + "_" + model + "_absolute.csv")
        trajs.append(traj)
    
    # plot_route(trajectories = trajs, models = models ,labels = ['Ground Truth', 'DeepVO', 'DeepVO (SE3)', 'DeepVO (Quat)'], colors = ['darkseagreen', 'tomato', 'steelblue', 'orange'])
    plot_route_XZ(trajectories = trajs, title=traj_name,models = models ,labels = ['Ground Truth', 'original', 'SE3', 'quaternion Euclidean', 'quaternion geodesic'], colors = ['gray','steelblue','darkseagreen', 'tomato', 'orange'  ])
    trajs = []
